In [1]:
#import dependencies
import pandas as pd
from pathlib import Path


In [2]:
#create reference to csv files (schools_complete.csv and students_complete.csv)
schools_csv = Path("Resources/schools_complete.csv")
students_csv = Path("Resources/students_complete.csv")

#read files and create dataframes
schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

#combine into one dataframe
merged_df = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])
merged_df.head()



,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
merged_df.columns


Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [22]:
#calculate the total number of unique schools; this block is working

school_count = len(schools_df)
school_count 



15

In [23]:
#calculate the total number of unique students; this block is working

student_count = len(students_df)
student_count


39170

In [25]:
#calculate the total budget; this block is working

total_budget = schools_df["budget"].sum()
total_budget


24649428

In [28]:
#calculate the average math score
avg_math = students_df["math_score"].mean()
avg_math


78.98537145774827

In [29]:
#calculate the average reading score
avg_read = students_df["reading_score"].mean()
avg_read


81.87784018381414

In [30]:
#use the following to calculate the precentage of students who passed math (math scores greater than or equal to 90)
passing_math_count = merged_df[(merged_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [31]:
#calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = merged_df[(merged_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [32]:
#use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = merged_df[
    (merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [41]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
#num of schools, total students, total budget, avg math score, avg reading score, % math pass, % reading pass, overall pass%

#this is not working yet- the summary pulls, but the formats need to be set (4.3, p.18)

district_summary = pd.DataFrame([{"Metric": "Total Schools", "Value": school_count},
                                {"Metric": "Total Students", "Value": student_count},
                                {"Metric": "Total Budget", "Value": total_budget},
                                {"Metric": "Average Math Score", "Value": avg_math},
                                {"Metric": "Average Reading Score", "Value": avg_read}, 
                                {"Metric": "% Passing Math", "Value": passing_math_percentage},
                                {"Metric": "% Passing Reading", "Value": passing_reading_percentage},
                                {"Metric": "% Overall Passing", "Value": overall_passing_rate}])


# Formatting
#district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
#district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Metric,Value
0,Total Schools,1.500000e+01
1,Total Students,3.917000e+04
2,Total Budget,2.464943e+07
3,Average Math Score,7.898537e+01
4,Average Reading Score,8.187784e+01
5,% Passing Math,7.498085e+01
6,% Passing Reading,8.580546e+01
7,% Overall Passing,6.517233e+01
